<a href="https://colab.research.google.com/github/ggoddll99/24-2ESAA/blob/main/240920_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%EC%95%99%EC%83%81%EB%B8%94%ED%95%99%EC%8A%B5%EA%B3%BC%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


직접 투표 : 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것이다. 이렇게 다수결 투표로 정해지는 분류기.

간접 투표 : 모든 분류기가 클래스의 확률을 예측할 수 있으면(즉, predict_proba() 메서드가 있으면), 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있다. 이를 간접 투표라고 한다. 이 방식은 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 높다.

**즉, 차이점은 직접 투표는 예측의 확률을 고려하지 않고 다수결의 방식을 사용하지만, 간접 투표는 확률을 고려한다. (평균 사용)**

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

학습률을 높인다.

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [2]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False, parser='auto')
X, y = mnist["data"], mnist["target"]

* 훈련 데이터: 60~80%
* 검증 데이터: 10~20%
* 테스트 데이터: 10~20%

In [3]:
from sklearn.model_selection import train_test_split

# train/valid/test dataset
# Step 1: 먼저 데이터셋을 훈련 데이터와 나머지(검증+테스트) 데이터로 나눕니다.
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

# Step 2: 나머지 데이터를 다시 검증 데이터와 테스트 데이터로 나눕니다.
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다.

In [6]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

In [7]:
# model fitting
rfc=RandomForestClassifier(n_estimators=100, random_state=42)
etc=ExtraTreesClassifier(n_estimators=100, random_state=42)
svc=SGDClassifier(loss='hinge', n_jobs=-1, max_iter=1000, random_state=42)
mlp=MLPClassifier(random_state=42)

rfc.fit(X_train, y_train)
etc.fit(X_train, y_train)
svc.fit(X_train, y_train)
mlp.fit(X_train, y_train)

MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [8]:
# model fitting
from sklearn.ensemble import VotingClassifier

voting_clf=VotingClassifier(
    estimators=[('rfc', rfc), ('etc', etc), ('svc', svc), ('mlp', mlp)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rfc', RandomForestClassifier(random_state=42)),
                             ('etc', ExtraTreesClassifier(random_state=42)),
                             ('svc', SGDClassifier(n_jobs=-1, random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [9]:
from sklearn.metrics import accuracy_score

y_pred = voting_clf.predict(X_val)  # 검증 데이터에 대한 예측
accuracy = accuracy_score(y_val, y_pred)  # 정확도 계산
print(f"Voting Classifier Accuracy: {accuracy:.4f}")

Voting Classifier Accuracy: 0.9681


####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [10]:
# 각 분류 모델 학습
# 각 모델의 검증 데이터셋에서 성능 평가
score1 = rfc.score(X_val, y_val)
score2 = etc.score(X_val, y_val)
score3 = svc.score(X_val, y_val)
score4 = mlp.score(X_val, y_val)

In [11]:
# 각 분류 모델의 성능 확인
print(f"rfc Accuracy: {score1:.4f}")
print(f"etc Accuracy: {score2:.4f}")
print(f"svc Accuracy: {score3:.4f}")
print(f"mlp Accuracy: {score4:.4f}")

# 성능이 가장 낮은 모델
scores = {'rfc': score1, 'etc': score2, 'svc': score3, 'mlp': score4}
min_model = min(scores, key=scores.get)  # 가장 낮은 성능을 가진 모델
print(f"Removing the lowest performing model: {min_model}")

rfc Accuracy: 0.9658
etc Accuracy: 0.9683
svc Accuracy: 0.8779
mlp Accuracy: 0.9591
Removing the lowest performing model: svc


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. svc

In [12]:
# 가장 성능이 낮은 모델 제거
estimators = [('rfc', rfc), ('etc', etc), ('svc', svc), ('mlp', mlp)]
scores = [score1, score2, score3, score4]

# 가장 낮은 성능의 모델 찾기
min_index = scores.index(min(scores))  # 성능이 가장 낮은 모델의 인덱스
print(f"Removing the lowest performing model: {estimators[min_index][0]}")
del estimators[min_index]

# 모델 제거 후 VotingClassifier 다시 생성
voting_clf_reduced = VotingClassifier(estimators=estimators, voting='hard')

Removing the lowest performing model: svc


In [13]:
# model fitting
voting_clf_reduced.fit(X_train, y_train)

VotingClassifier(estimators=[('rfc', RandomForestClassifier(random_state=42)),
                             ('etc', ExtraTreesClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [14]:
# 모델 제거 후 성능 확인
reduced_score = voting_clf_reduced.score(X_val, y_val)
print(f"Voting Classifier (Reduced) Accuracy: {reduced_score:.4f}")

Voting Classifier (Reduced) Accuracy: 0.9702


### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [16]:
# 검증 세트에서 각 모델로 예측 수행
pred1 = rfc.predict(X_val).reshape(-1, 1)
pred2 = etc.predict(X_val).reshape(-1, 1)
pred3 = svc.predict(X_val).reshape(-1, 1)
pred4 = mlp.predict(X_val).reshape(-1, 1)

# 각 모델의 예측 결과를 열로 쌓아 새로운 훈련 데이터 생성
stacked_predictions = np.hstack((pred1, pred2, pred3, pred4))

In [17]:
# 검증 데이터의 실제 타깃 값(y_val)을 새로운 훈련 세트의 타깃으로 사용
new_train_y = y_val

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [19]:
rfc_stck=RandomForestClassifier(n_estimators=100, random_state=42)

# 랜덤 포레스트 모델을 새로운 훈련 세트(stacked_predictions)로 학습
rfc_stck.fit(stacked_predictions, new_train_y)

RandomForestClassifier(random_state=42)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다.

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [20]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
rf_train_pred = rfc_stck.predict(stacked_predictions)

In [21]:
# model test
accuracy = accuracy_score(new_train_y, rf_train_pred)
print(f"Random Forest Classifier Accuracy on Stacked Predictions: {accuracy:.4f}")

Random Forest Classifier Accuracy on Stacked Predictions: 0.9818
